In [1]:
import qiskit

from qiskit_aer import AerSimulator

from qiskit_ibm_runtime.fake_provider import fake_backend

In [2]:
import numpy as np

import warnings

from importlib.util import find_spec

import qiskit

from qiskit.qasm2 import dumps
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_aer import AerSimulator

In [3]:
try:
    import bqskit

except ModuleNotFoundError:
    warnings.warn("BQSKit not found", ImportWarning)

try:
    import pytket
    import pytket.extensions.qiskit

except ModuleNotFoundError:
    warnings.warn("Pytket not found", ImportWarning)

In [4]:
%cd -q ../..

from rivet_transpiler import transpile

In [5]:
backend = AerSimulator()

In [6]:
isinstance(backend, (qiskit.providers.BackendV1,
                     fake_backend.FakeBackend))

False

In [7]:
isinstance(backend, (qiskit.providers.BackendV2,
                     fake_backend.FakeBackendV2))

True

In [8]:
QUBITS_COUNT = 5

def get_ghz_circuit(qubits_count):

    qubits = list(range(qubits_count))

    circuit = qiskit.QuantumCircuit(qubits_count)

    circuit.h(0)

    for control, target in zip(qubits, qubits[1:]):

        circuit.cx(control, target)

    return circuit

ghz_circuit = get_ghz_circuit(QUBITS_COUNT)

ghz_circuit.draw()

┌───┐                    
q_0: ┤ H ├──■─────────────────
     └───┘┌─┴─┐               
q_1: ─────┤ X ├──■────────────
          └───┘┌─┴─┐          
q_2: ──────────┤ X ├──■───────
               └───┘┌─┴─┐     
q_3: ───────────────┤ X ├──■──
                    └───┘┌─┴─┐
q_4: ────────────────────┤ X ├
                         └───┘

In [9]:
transpiled_ghz_circuit = transpile(
    ghz_circuit,
    backend=backend
)

transpiled_ghz_circuit.draw()

┌───┐                    
q_0: ┤ H ├──■─────────────────
     └───┘┌─┴─┐               
q_1: ─────┤ X ├──■────────────
          └───┘┌─┴─┐          
q_2: ──────────┤ X ├──■───────
               └───┘┌─┴─┐     
q_3: ───────────────┤ X ├──■──
                    └───┘┌─┴─┐
q_4: ────────────────────┤ X ├
                         └───┘

In [10]:
def model_from_ibmq_backend(backend):

    """
    Create a machine model from an IBMQ backend.

    Args:
        backend: The IBMQ backend.

    Returns:
        MachineModel: The machine model representing the backend.
    """

    # Based on https://github.com/BQSKit/bqskit/blob/main/bqskit/ext/qiskit/models.py

    IBMQ_BASIS_GATES_LIMIT = 10

    # Backend

    if backend is None:
        ibmq_backend = AerSimulator()
    else:
        ibmq_backend = backend

    if isinstance(ibmq_backend, (qiskit.providers.BackendV1,
                                 fake_backend.FakeBackend)):

        # print('IBMQ Backend Version 1')

        qubits_count = ibmq_backend.configuration().n_qubits
        basis_gates = ibmq_backend.configuration().basis_gates
        coupling_map = ibmq_backend.configuration().coupling_map

    if isinstance(ibmq_backend, (qiskit.providers.BackendV2,
                                 fake_backend.FakeBackendV2)):

        # print('IBMQ Backend Version 2')

        qubits_count = ibmq_backend.target.num_qubits
        basis_gates = ibmq_backend.target.operation_names
        coupling_map = ibmq_backend.target.build_coupling_map()

    # Gate Set

    gate_dict = {'cx': bqskit.ir.gates.CNOTGate(),
                 'cz': bqskit.ir.gates.CZGate(),
                 'u3': bqskit.ir.gates.U3Gate(),
                 'u2': bqskit.ir.gates.U2Gate(),
                 'u1': bqskit.ir.gates.U1Gate(),
                 'rz': bqskit.ir.gates.RZGate(),
                 'sx': bqskit.ir.gates.SXGate(),
                 'x': bqskit.ir.gates.XGate(),
                 'p': bqskit.ir.gates.RZGate()}

    if len(basis_gates) > IBMQ_BASIS_GATES_LIMIT:

        gate_set = {bqskit.ir.gates.CNOTGate(),
                    bqskit.ir.gates.RZGate(),
                    bqskit.ir.gates.SXGate()}
    else:

        gate_set = {gate_dict.get(basis_gate)
                    for basis_gate in basis_gates} - {None}

    # Coupling List

    coupling_list = None

    if coupling_map is not None:

        coupling_list = list({tuple(sorted(edge)) for edge in coupling_map})

    # Mashine Model

    machine_model = bqskit.MachineModel(qubits_count, coupling_list, gate_set)

    return machine_model

In [11]:
model_from_ibmq_backend(backend)